Author: Martin Loza

Date: 23/12/28

In this workflow I want to explore the downloaded data from 4DN. I want to know if they include the normalized counts. 



In [70]:
# Init libraries
import cooler
import numpy as np
import pandas as pd
import h5py

# Global variables
data_dir = "/mnt/d/Projects/HK_Interactions/Data/HiC/4DN_portal/"
out_dir = "/mnt/c/Users/Marti/Documents/Projects/HK_Interactions/Analysis/2024_01/2024_01_09/Results/"

# Local functions

def GetResolutionsMcool(mcool_path):
    """
    Get the available resolutions from an mcool file.
    
    Parameters:
    - mcool_path: str, path to the mcool file
    
    Returns:
    - resolutions: list of int, available resolutions in the mcool file
    """
    # Open the mcool file
    with h5py.File(mcool_path, 'r') as tmp_mcool:
        # Get the available resolutions
        resolutions = tmp_mcool['resolutions'].keys()
        # Change the resolutions to a vector
        resolutions = [int(res) for res in resolutions]
        # Order the resolutions
        resolutions.sort()
    
    return resolutions

def SameResolutionMcools(mcools_path):
    """
    Compare the resolutions of multiple mcool files.
    
    Parameters:
    - mcool_files: list of str, paths to the mcool files
    
    Returns:
    - resolutions_equal: bool, True if all resolutions are equal, False otherwise
    """
    resolutions_list = []
    
    for mcool_file in mcools_path:
        resolutions = GetResolutionsMcool(mcool_file)
        resolutions_list.append(resolutions)
    
    resolutions_equal = all(resolutions_list[i] == resolutions_list[i+1] for i in range(len(resolutions_list)-1))
    
    return resolutions_equal

In [71]:
mcool_file = "4DNFII4IPDTR.mcool"
GetResolutionsMcool(data_dir + mcool_file)

[1000,
 2000,
 5000,
 10000,
 25000,
 50000,
 100000,
 250000,
 500000,
 1000000,
 2500000,
 5000000,
 10000000]

In [72]:
# List of mcool files
mcool_files = [
    data_dir +"4DNFII4IPDTR.mcool",
    data_dir + "4DNFIU5DRADP.mcool"
]

SameResolutionMcools(mcool_files)

True

In [73]:
resolution = 1000000

i = 0
# Get a cooler file
current_cooler = cooler.Cooler(mcool_files[i] + "::resolutions/" + str(resolution))
# Get the matrix
current_matrix = current_cooler.matrix(balance=True, as_pixels=True, join=True)[:]
# Remove interchromosomal interactions
current_matrix = current_matrix[current_matrix['chrom1'] == current_matrix['chrom2']]

In [74]:
current_matrix

,chrom1,start1,end1,chrom2,start2,end2,count,balanced
0,chr1,0,1000000,chr1,0,1000000,5855,NaN
1,chr1,0,1000000,chr1,1000000,2000000,859,NaN
2,chr1,0,1000000,chr1,2000000,3000000,93,NaN
3,chr1,0,1000000,chr1,3000000,4000000,68,NaN
4,chr1,0,1000000,chr1,4000000,5000000,31,NaN
...,...,...,...,...,...,...,...,...
4192948,chrY,23000000,24000000,chrY,26000000,27000000,1,NaN
4192949,chrY,25000000,26000000,chrY,25000000,26000000,2,NaN
4192950,chrY,26000000,27000000,chrY,26000000,27000000,1020,NaN
4192951,chrY,26000000,27000000,chrY,56000000,57000000,20,NaN


With these test and functions we are ready to apply them in a loop for all the celltypes and available resolutions. Let's make that in another notebook

## Tests

### Are data normalized? is it ICE normalization?

Conclusions: Yes, they have several normalizations as VR, KR, etc. We can use the ICE normalized data.

In [40]:
# Load an cool file
cool_data = cooler.Cooler(mcool_files[0]+ "::/resolutions/1000000")

# Get the bins
bins = cool_data.bins()[:]
bins.head()

,chrom,start,end,KR,VC,VC_SQRT,weight
0,chr1,0,1000000,0.395594,0.236638,0.431676,NaN
1,chr1,1000000,2000000,1.009324,1.237803,0.987282,0.005064
2,chr1,2000000,3000000,1.027329,1.325048,1.021483,0.005795
3,chr1,3000000,4000000,1.213692,1.762398,1.178060,0.004491
4,chr1,4000000,5000000,1.078945,1.449865,1.068511,0.005177


In [42]:
# get the pixels as a dataframe
cool_m = cool_data.matrix(balance=True, as_pixels=True, join=True)[:]

In [58]:
cool_m[2590:]

,chrom1,start1,end1,chrom2,start2,end2,count,balanced
2590,chr1,0,1000000,chrY,19000000,20000000,1,NaN
2591,chr1,0,1000000,chrY,56000000,57000000,1,NaN
2592,chr1,1000000,2000000,chr1,1000000,2000000,34144,0.875420
2593,chr1,1000000,2000000,chr1,2000000,3000000,1608,0.047183
2594,chr1,1000000,2000000,chr1,3000000,4000000,434,0.009869
...,...,...,...,...,...,...,...,...
4192948,chrY,23000000,24000000,chrY,26000000,27000000,1,NaN
4192949,chrY,25000000,26000000,chrY,25000000,26000000,2,NaN
4192950,chrY,26000000,27000000,chrY,26000000,27000000,1020,NaN
4192951,chrY,26000000,27000000,chrY,56000000,57000000,20,NaN


Great they are normalized!! we can apply the same workflow as before